In [7]:
'''
網路分析 (network analysis)
'''
import math, random, re
from collections import defaultdict, Counter, deque
# from linear_algebra import dot, get_row, get_column, make_matrix, magnitude, scalar_multiply, shape, distance
from functools import partial

users = [
    { "id": 0, "name": "Hero" },
    { "id": 1, "name": "Dunn" },
    { "id": 2, "name": "Sue" },
    { "id": 3, "name": "Chi" },
    { "id": 4, "name": "Thor" },
    { "id": 5, "name": "Clive" },
    { "id": 6, "name": "Hicks" },
    { "id": 7, "name": "Devin" },
    { "id": 8, "name": "Kate" },
    { "id": 9, "name": "Klein" }
]

friendships = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
               (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]


# 把朋友列表加到每個使用者的 dict字典中
for user in users:
    user["friends"] = []
    
for i, j in friendships:
    # users[i] 就是 id 為 i 的使用者
    users[i]["friends"].append(users[j])    # 把 j 加為 i 的朋友
    users[j]["friends"].append(users[i])    # 把 i 加為 j 的朋友

In [10]:
# 利用 frontier 的佇列 (queue)
# 存放要探索的使用者資料
from collections import deque

def shortest_paths_from(from_user):
    # 一個 dict 字典: 鍵值是使用者編號，
    # 相應的值則是「所有」其他各使用者連往此使用者的最短路徑
    shoortest_path_to = { from_user["id"] : [[]]}
    
    # 存放 (prev_user, user) 的佇列，稍後在針對其中每個元素進行檢查
    # 一開始把所有 (from_user, from_user的朋友) 全放進佇列中
    frontier = deque((from_user, friend)
                    for friend in from_user["friends"])
    
    # 一直做到佇列完全空了為止
    while frontier:
        prev_user, user = frontier.popleft()    # 移出佇列中的
        user_id = user["id"]                    # 第一筆使用者資料
        
        # 考慮到把 (prev_user, user) 送進佇列的作法
        # 必然已經知道一些連往 prev_user 的最短路徑
        paths_to_prev_user = shortest_paths_to[prev_user["id"]]
        new_paths_to_user = [path + [user_id] for path in paths_to_prev_user]
        
        # 有可能已經知道一條最短路徑
        old_paths_to_user = shortest_paths_to.get(user_id, [])
        
        # 到目前為止所看到的最短路徑有多長？
        if old_paths_to_user:
            min_path_length = len(old_paths_to_user[0])
        else:
            min_path_length = float('inf')
            
        # 不能比舊路徑長，而且必須是新路徑，才會被保留
        new_paths_to_user = [path
                            for path in new_paths_to_user
                            if len(path) <= min_path_length
                            and path not in old_paths_to_user]
        
        shortest_paths_to[user_id] = old_paths_to_user + new_paths_to_user
        
        # 把沒看過的相鄰節點，添加到 frontier 中
        frontier.extend((user, friend)
                       for friend in user["friends"]
                       if friend["id"] not in shortest_paths_to)
    return shortest_paths_to

In [12]:
# 把每個節點相應的 dict 保存
for user in users:
    user["shortest_paths"] = shortest_paths_from(user)
    
# 在每條最短路徑上，為每個節點增加 1/n 的中心度
for user in users:
    user["betweenness_centrality"] = 0.0
    
for source in users:
    source_id = source["id"]
    
for target_id, paths in source["shortest_paths"].items():
    if source_id < target_id:        # 不要重複計算
        num_paths = len(paths)
        contrib = 1 / num_paths
        for path in paths:
            for id in path:
                if id not in [source_id, target_id]:
                    users[id]["betweenness_centrality"] += contrib

NameError: name 'shortest_paths_to' is not defined

In [13]:
# 緊密中心度 (closeness centrality)
def farness(user):
    '''
    與所有其他使用者之間最短路徑的長度總和
    '''
    return sum(len(paths[0])
              for paths in user["shortest_paths"].values())

for user in users:
    user["closeness_centrality"] = 1 / farness(user)

KeyError: 'shortest_paths'

In [15]:
'''
特徵向量中心度 (eigenvector centrality)
'''
# 矩陣乘法 或 乘積
def matrix_product_entry(A, B, i, j):
    return dot(get_row(A, i), get_column(B, j))

def matrix_multiply(A, B):
    n1, k1 = shape(A)
    n2, k2 = shape(B)
    if k1 != n2:
        raise ArithmeticError("incompatible shapes!")
    return make_matrix(n1, k2, partial(matrix_product_entry, A, B))

def vector_as_matrix(v):
    '''
    把一個 (以列表來表示) 向量 v 轉換成一個 n * 1 矩陣
    '''
    return [[v_i] for v_i in v]

def vector_from_matrix(v_as_matrix):
    '''
    把一個 n * 1 矩陣轉換成一個數值列表
    '''
    return [row[0] for row in v_as_matrix]

def matrix_operate(A, v):
    v_as_matrix = vector_as_matrix(v)
    product = matrix_multiply(A, v_as_matrix)
    return vector_from_matrix(product)

In [16]:
# 找出 A 的特徵向量套入矩陣運算
# 把結果調整成向量長度為 1 的單位向量
def find_eigenvector(A, tolenrance=0.00001):
    guess = [random.random() for __ in A]
    
    while True:
        result = matrix_operate(A, guess)
        length = magnitude(result)
        next_guess = scalar_multiply(1/length, result)
        
        if distance(guess, next_guess) < tolerance:
            return next_guess, length    # 特徵向量, 特徵值
        
        guess = next_guess

In [17]:
'''
中心度 (centrality)
'''
def entry_fn(i, j):
    return 1 if (i, j) in friendships or (j, i) in friendships else 0

n = len(users)
adjacency_matrix = make_matrix(n, n, entry_fn)

# 每個使用者所對應的向量元素值 
eigenvector_centralities, _ = find_eigenvector(adjacency_matrix)

# 用一個純亮去乘以 eigenvector_centralities
matrix_operate(adjacency_matrix, eigenvector_centralities)

# 製造出第 i 個元素
dot(get_row(adjacency_matrix, i), eigenvector_centralities)

NameError: name 'make_matrix' is not defined

In [18]:
'''
有向圖 (directed graphs)
'''
endorsements = [(0, 1), (1, 0), (0, 2), (2, 0), (1, 2), (2, 1), (1, 3),
                (2, 3), (3, 4), (5, 4), (5, 6), (7, 5), (6, 8), (8, 7), (8, 9)]

for user in users:
    user["endorses"] = []       # 添加一個列表，以追蹤向外的認同關係
    user["endorsed_by"] = []    # 另一個追蹤向內的被認同關係

for source_id, target_id in endorsements:
    users[source_id]["endorses"].append(users[target_id])
    users[target_id]["endorsed_by"].append(users[source_id])


endorsements_by_id = [(user["id"], len(user["endorsed_by"]))
                      for user in users]

sorted(endorsements_by_id,
       key=lambda pair: pair[1],
       reverse=True)

[(0, 2),
 (1, 2),
 (2, 2),
 (3, 2),
 (4, 2),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

In [23]:
# 將每個節點的頁面級別分數，未分配的部分平均分給所有節點
def page_rank(users, damping = 0.85, num_iters = 100):
    # 一開始先平均分配頁面級別分數
    num_users = len(users)
    pr = { user["id"] : 1 / num_users for user in users }
    
    # 頁面級別分數其中的一小部分比例
    # 所有節點都會反覆取得這一部分的分數
    base_pr = (1 - damping) / num_users
    
    for __ in range(num_iters):
        next_pr = { user["id"] : base_pr for user in users }
        for user in users:
            # 將頁面級別分數分配給向外連的節點
            links_pr = pr[user["id"]] * damping
            for endorsee in user["endorses"]:
                next_pr[endorsee["id"]] += link_pr / len(user["endorses"])
                pr = next_pr
            return pr